In [7]:
import pandas as pd
import numpy as np

def price_storage_contract(
    file_path,
    injection_dates,
    withdrawal_dates,
    injection_rate,
    withdrawal_rate,
    max_storage,
    storage_cost_per_unit_per_month
):
    
    # =====================
    # Load & Prepare Data
    # =====================
    
    df = pd.read_csv(file_path)
    
    # Clean column names (in case of spaces)
    df.columns = df.columns.str.strip()
    
    # Convert date column
    df['Dates'] = pd.to_datetime(df['Dates'])
    
    # Convert price column
    df['Prices'] = df['Prices'].astype(float)
    
    # Set index
    df.set_index('Dates', inplace=True)
    df.sort_index(inplace=True)
    
    
    # =====================
    # Convert Inputs
    # =====================
    
    injection_dates = pd.to_datetime(injection_dates)
    withdrawal_dates = pd.to_datetime(withdrawal_dates)
    
    inventory = 0
    total_cashflow = 0
    
    all_dates = sorted(list(set(injection_dates) | set(withdrawal_dates)))
    
    inventory_history = {}
    
    
    # =====================
    # Simulation
    # =====================
    
    for date in all_dates:
        
        if date not in df.index:
            raise ValueError(f"Price not available for {date}")
        
        price = df.loc[date, 'Prices']
        
        # Injection
        if date in injection_dates:
            inject_volume = min(injection_rate, max_storage - inventory)
            inventory += inject_volume
            total_cashflow -= inject_volume * price
        
        # Withdrawal
        if date in withdrawal_dates:
            withdraw_volume = min(withdrawal_rate, inventory)
            inventory -= withdraw_volume
            total_cashflow += withdraw_volume * price
        
        inventory_history[date] = inventory
    
    
    # =====================
    # Storage Cost
    # =====================
    
    for inv in inventory_history.values():
        total_cashflow -= inv * storage_cost_per_unit_per_month
    
    
    return total_cashflow

contract_value = price_storage_contract(
    file_path="C:\\Users\\91930\\Downloads\\Quant finance jp morgan\\task 2\\Nat_Gas.csv",
    injection_dates=["2021-01-31", "2021-02-28", "2021-03-31"],
    withdrawal_dates=["2021-10-31", "2021-11-30"],
    injection_rate=1000,
    withdrawal_rate=1000,
    max_storage=3000,
    storage_cost_per_unit_per_month=0.05
)

print("Storage Contract Value:", round(contract_value, 2))


Storage Contract Value: -11850.0


C:\Users\91930\AppData\Local\Temp\ipykernel_11212\1913262037.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


In [9]:
import pandas as pd
import numpy as np


def price_storage_contract(
    file_path,
    injection_dates,
    withdrawal_dates,
    injection_rate,
    withdrawal_rate,
    max_storage,
    storage_cost_per_unit_per_month
):
    
    # ==========================
    # 1. Load & Clean Data
    # ==========================
    
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    
    df['Dates'] = pd.to_datetime(df['Dates'])
    df['Prices'] = df['Prices'].astype(float)
    
    df.set_index('Dates', inplace=True)
    df.sort_index(inplace=True)
    
    
    # ==========================
    # 2. Prepare Inputs
    # ==========================
    
    injection_dates = pd.to_datetime(injection_dates)
    withdrawal_dates = pd.to_datetime(withdrawal_dates)
    
    inventory = 0
    total_cashflow = 0
    
    # Create full monthly timeline
    start_date = min(injection_dates.min(), withdrawal_dates.min())
    end_date = max(injection_dates.max(), withdrawal_dates.max())
    
    timeline = df.loc[start_date:end_date].index
    
    # Cashflow tracker
    results = []
    
    
    # ==========================
    # 3. Month-by-Month Simulation
    # ==========================
    
    for date in timeline:
        
        price = df.loc[date, 'Prices']
        injection = 0
        withdrawal = 0
        cashflow = 0
        
        # Injection decision
        if date in injection_dates:
            injection = min(injection_rate, max_storage - inventory)
            inventory += injection
            cashflow -= injection * price
        
        # Withdrawal decision
        if date in withdrawal_dates:
            withdrawal = min(withdrawal_rate, inventory)
            inventory -= withdrawal
            cashflow += withdrawal * price
        
        # Storage cost (applied every month)
        storage_cost = inventory * storage_cost_per_unit_per_month
        cashflow -= storage_cost
        
        total_cashflow += cashflow
        
        results.append([
            date, price, injection, withdrawal,
            inventory, storage_cost, cashflow
        ])
    
    
    # ==========================
    # 4. Return Detailed Output
    # ==========================
    
    results_df = pd.DataFrame(results, columns=[
        "Date", "Price", "Injection",
        "Withdrawal", "End_Inventory",
        "Storage_Cost", "Net_Cashflow"
    ])
    
    return total_cashflow, results_df

value, table = price_storage_contract(
    file_path="C:\\Users\\91930\\Downloads\\Quant finance jp morgan\\task 2\\Nat_Gas.csv",
    injection_dates=["2021-01-31", "2021-02-28", "2021-03-31"],
    withdrawal_dates=["2021-10-31", "2021-11-30"],
    injection_rate=1000,
    withdrawal_rate=1000,
    max_storage=3000,
    storage_cost_per_unit_per_month=0.05
)

print("Contract Value:", round(value, 2))
print(table)


Contract Value: -12750.0
         Date  Price  Injection  Withdrawal  End_Inventory  Storage_Cost  \
0  2021-01-31  10.90       1000           0           1000          50.0   
1  2021-02-28  10.90       1000           0           2000         100.0   
2  2021-03-31  10.90       1000           0           3000         150.0   
3  2021-04-30  10.40          0           0           3000         150.0   
4  2021-05-31   9.84          0           0           3000         150.0   
5  2021-06-30  10.00          0           0           3000         150.0   
6  2021-07-31  10.10          0           0           3000         150.0   
7  2021-08-31  10.30          0           0           3000         150.0   
8  2021-09-30  10.20          0           0           3000         150.0   
9  2021-10-31  10.10          0        1000           2000         100.0   
10 2021-11-30  11.20          0        1000           1000          50.0   

    Net_Cashflow  
0       -10950.0  
1       -11000.0  
2    

C:\Users\91930\AppData\Local\Temp\ipykernel_11212\1756769776.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])
